In [1]:
import numpy as np
import pandas as pd
import sqlite3

import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False 

import jieba
from snownlp import SnowNLP
from wordcloud import WordCloud

In [2]:
conn = sqlite3.connect('netease_cloud_music.db')

In [3]:
sql = '''
    SELECT *
    FROM song
    WHERE artists LIKE '%窦唯%'
'''

In [4]:
song = pd.read_sql(sql, con=conn)

In [5]:
song.nunique()

song_id       445
song_name     358
artists        13
album_name     75
album_id       75
duration      401
dtype: int64

In [6]:
song.sample(5)

,song_id,song_name,artists,album_name,album_id,duration
351,77003,Untitled 4,"窦唯,FM3,",后观音,7565,227709
71,77305,窗外,"窦唯,",希望之光精选辑,7595,285466
344,26032093,伍 西夕中东,"窦唯,暮良文王,",相相生,2360539,107000
196,76882,Track 05,"窦唯,",五音环乐,7545,737672
275,26577380,望天儿,"窦唯,不一定,",五鹊六雁,2523295,43000


In [7]:
sql = '''
    SELECT *
    FROM comment
    WHERE song_id IN (
        SELECT song_id
        FROM song
        WHERE artists LIKE '%窦唯%'
    )
'''

In [8]:
comment = pd.read_sql(sql, con=conn)

In [9]:
comment.nunique()

comment_id       159232
user_id           70254
user_nickname     68798
user_avatar       80094
content          136898
likeCount           616
song_id             445
dtype: int64

In [10]:
song_top10_num = comment.groupby('song_id').size().sort_values(ascending=False)[0:10]
song_top10 = song[song['song_id'].isin(song_top10_num.index)].iloc[:, 0:2]
song_top10['num'] =  song_top10_num.tolist()
print(song_top10)

       song_id song_name    num
0      5279713      高级动物  11722
4      5279715      悲伤的梦   9316
5        77169      暮春秋色   7464
8      5279714       噢 乖   6477
13   526468453    送别2017   5605
28   512298988      重返魔域   4677
124   27853979       殃金咒   4493
327   26031014        雨吁   3965
377   34248413   既然我们是兄弟   3845
435   28465036       天宫图   3739


In [11]:
user_top10 = comment.groupby('user_id').size().sort_values(ascending=False)[0:10]
print(user_top10)

user_id
42830600     549
33712056     322
51625217     273
284151966    242
2159884      234
271253793    234
388206024    233
263344124    232
84030184     209
131005965    204
dtype: int64


In [12]:
comment_top10 = comment.sort_values(['likeCount'], ascending=False)[0:10]
print(comment_top10[['comment_id', 'likeCount']])

        comment_id  likeCount
11252     51694054      35285
10522    133265373      15409
10211    148045985      12886
146129    40249220       9234
10038    157500246       7670
38728      6107434       7393
48826    658314395       5559
31101      7875585       5248
146213    35287069       4900
37307    231408710       4801


In [13]:
#comment['semiscore'] = comment['content'].apply(lambda x: SnowNLP(x).sentiments)
#comment['semilabel'] = comment['semiscore'].apply(lambda x: 1 if x > 0.5 else -1)

In [17]:
comment.to_csv('douwei.csv', index=False)